In [11]:
import os
import logging
import time
import glob
from shutil import copyfile
from multiprocessing import Pool, Value, Lock
import openslide
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import pandas as pd
import logging
import time
import glob
from shutil import copyfile

# Importing defaultdict 
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from operator import itemgetter 

In [18]:
ROOT_PATH = '/media/mak/mirlproject1'
# ROOT_PATH = '/media/balaji/CamelyonProject/CAMELYON_DATASET'

# Camelyon 2016
train_tif_cm16_path = ROOT_PATH+'/CAMELYON16/TrainingData/normal_tumor'
train_xml_cm16_path = ROOT_PATH+'/CAMELYON16/TrainingData/lesion_annotations'
train_mask_cm16_path = ROOT_PATH+'/CAMELYON16/TrainingData/lesion_masks'

test_tif_cm16_path = ROOT_PATH+'/CAMELYON16/Testset/Images'
test_xml_cm16_path = ROOT_PATH+'/CAMELYON16/Testset/lesion_annotations'
test_mask_cm16_path = ROOT_PATH+'/CAMELYON16/Testset/Backup/Masks/tif_files'
test_reference_cm16_path = ROOT_PATH+'/CAMELYON16/Testset/reference.csv'

In [19]:
# Non-exhaustive Annotations of Camelyon16
# 'tumor_079': Blurred tumor region is not annotated.
# 'tumor_092': Blurred tumor region on the adjacent tissue is not annotated.
# TODOs to check:
# test_049: Duplicate slide.
# test_114: Does not have exhaustive annotations.
train_tumor_nea_file_list = ['Tumor_010.tif','Tumor_015.tif','Tumor_018.tif','Tumor_020.tif',
                             'Tumor_025.tif','Tumor_029.tif','Tumor_033.tif','Tumor_034.tif',
                             'Tumor_044.tif','Tumor_046.tif','Tumor_051.tif','Tumor_054.tif',
                             'Tumor_055.tif','Tumor_056.tif','Tumor_067.tif','Tumor_079.tif',
                             'Tumor_085.tif','Tumor_092.tif','Tumor_095.tif','Tumor_110.tif']

test_tumor_nea_file_list = ['Test_049.tif','Test_114.tif']

In [20]:
# List of CM17 training tumor cases with annotation
train_tumor_cm17_annotated_file_list = ['patient_024_node_1_mask.tif', 'patient_052_node_1_mask.tif',
                              'patient_004_node_4_mask.tif', 'patient_009_node_1_mask.tif',
                              'patient_010_node_4_mask.tif', 'patient_012_node_0_mask.tif',
                              'patient_015_node_1_mask.tif', 'patient_015_node_2_mask.tif', 
                              'patient_016_node_1_mask.tif', 'patient_017_node_1_mask.tif', 
                              'patient_017_node_2_mask.tif', 'patient_017_node_4_mask.tif', 
                              'patient_020_node_2_mask.tif', 'patient_020_node_4_mask.tif', 
                              'patient_021_node_3_mask.tif', 'patient_022_node_4_mask.tif',
                              'patient_024_node_2_mask.tif', 'patient_034_node_3_mask.tif', 
                              'patient_036_node_3_mask.tif', 'patient_038_node_2_mask.tif', 
                              'patient_039_node_1_mask.tif', 'patient_040_node_2_mask.tif', 
                              'patient_041_node_0_mask.tif', 'patient_042_node_3_mask.tif', 
                              'patient_044_node_4_mask.tif', 'patient_045_node_1_mask.tif', 
                              'patient_046_node_3_mask.tif', 'patient_046_node_4_mask.tif', 
                              'patient_048_node_1_mask.tif', 'patient_051_node_2_mask.tif', 
                              'patient_060_node_3_mask.tif', 'patient_061_node_4_mask.tif',
                              'patient_062_node_2_mask.tif', 'patient_064_node_0_mask.tif', 
                              'patient_066_node_2_mask.tif', 'patient_067_node_4_mask.tif',
                              'patient_068_node_1_mask.tif', 'patient_072_node_0_mask.tif',
                              'patient_073_node_1_mask.tif', 'patient_075_node_4_mask.tif', 
                              'patient_080_node_1_mask.tif', 'patient_081_node_4_mask.tif',
                              'patient_086_node_0_mask.tif', 'patient_086_node_4_mask.tif', 
                              'patient_087_node_0_mask.tif', 'patient_088_node_1_mask.tif',
                              'patient_089_node_3_mask.tif', 'patient_092_node_1_mask.tif', 
                              'patient_096_node_0_mask.tif', 'patient_099_node_4_mask.tif']

In [21]:
# Camelyon 2017
train_tif_cm17_path = ROOT_PATH+'/CAMELYON17/training/dataset'
train_xml_cm17_path = ROOT_PATH+'/CAMELYON17/training/groundtruth/lesion_annotations/XML'
train_mask_cm17_path = ROOT_PATH+'/CAMELYON17/training/groundtruth/lesion_annotations/Mask'
# train_stage_labels_path = ROOT_PATH+'/CAMELYON17/training/groundtruth/stage_labels.csv'
train_stage_labels_path = '/media/mak/mirlproject1'+'/CAMELYON17/training/groundtruth/stage_labels.csv'


In [22]:
# Save Path
save_path = '../../datasetgen'
if not os.path.exists(save_path):
    os.makedirs(save_path)

### CM17 CV SPLITs

In [23]:
# Collect all CM17 WSI_paths
cm17_train_stage_labels_df = pd.read_csv(train_stage_labels_path)
# print (xml_file_list)
negative_list = []
itc_list = []
micro_list = []
macro_list = []

negative_list_mask = []
itc_list_mask = []
micro_list_mask = []
macro_list_mask = []

class_mapping = {'negative': 0, 'micro':1, 'macro':2, 'itc':3}

for index, row in cm17_train_stage_labels_df.iterrows():
    patient_name = row["patient"].split('.')[0]
    center_folder = 'center_'+str(int(patient_name.split('_')[1])//20)        
    if row["stage"] == 'negative':
        negative_list.append(os.path.join(train_tif_cm17_path, center_folder, row["patient"]))
        negative_list_mask.append(0)

    mask_file_name = patient_name +'_mask.tif'        
    if mask_file_name in train_tumor_cm17_annotated_file_list:
#         print (check_name[:11])
        if row["stage"] == 'itc':
            itc_list.append(os.path.join(train_tif_cm17_path, center_folder, row["patient"]))
            itc_list_mask.append(os.path.join(train_mask_cm17_path, mask_file_name))
        elif row["stage"] == 'micro':
            micro_list.append(os.path.join(train_tif_cm17_path, center_folder, row["patient"]))
            micro_list_mask.append(os.path.join(train_mask_cm17_path, mask_file_name))
        elif row["stage"] == 'macro':
            macro_list.append(os.path.join(train_tif_cm17_path, center_folder, row["patient"]))                    
            macro_list_mask.append(os.path.join(train_mask_cm17_path, mask_file_name))                    

cm17_train_patient_list = negative_list + micro_list + macro_list + itc_list
cm17_train_patient_mask_list = negative_list_mask + micro_list_mask + macro_list_mask + itc_list_mask
cm17_patient_label_list = len(negative_list)*[0] + len(micro_list)*[1] + len(macro_list)*[2]  + len(itc_list)*[3]
print ('Total_number of CM17 cases with annotations', len(cm17_patient_label_list))

Total_number of CM17 cases with annotations 368


In [24]:
# # print(cm17_train_patient_list)
# # print(cm17_patient_label_list)
# # print(patient_dict)
# X_train, X_test, y_train, y_test = train_test_split(cm17_train_patient_list, cm17_patient_label_list,\
#                                                     test_size=0.3, random_state=0, stratify=patient_label_list)

# train_df = pd.DataFrame({'image':X_train,'stage':y_train})
# train_df.to_csv(os.path.join(save_path, 'camelyon17_train_dataset.csv'), index=False)
# valid_df = pd.DataFrame({'image':X_test,'stage':y_test})
# valid_df.to_csv(os.path.join(save_path, 'camelyon17_valid_dataset.csv'), index=False)

In [25]:
SPLITS = 3
cm17_save_dir = os.path.join(save_path, 'cm17_cross_val_splits')
if not os.path.exists(cm17_save_dir):
    os.makedirs(cm17_save_dir)
    
skf = StratifiedKFold(n_splits=SPLITS)
# skf.get_n_splits(cm17_train_patient_list, cm17_patient_label_list)
counter = 0
for train_index, test_index in skf.split(cm17_train_patient_list, cm17_patient_label_list):
#     print("TRAIN:", train_index, "TEST:", test_index)    
    image_train, image_test = itemgetter(*train_index)(cm17_train_patient_list), itemgetter(*test_index)(cm17_train_patient_list) 
    label_train, label_test = itemgetter(*train_index)(cm17_patient_label_list), itemgetter(*test_index)(cm17_patient_label_list) 
    mask_train, mask_test = itemgetter(*train_index)(cm17_train_patient_mask_list), itemgetter(*test_index)(cm17_train_patient_mask_list) 
    
    train_data = {'Image_Path': image_train, 'Label': label_train, 'Mask_Path': mask_train}
    df_train = pd.DataFrame(train_data)
    # print (df_train)
    df_train.to_csv(os.path.join(cm17_save_dir, 'training_fold_{0}.csv'.format(counter)), index = False)
    validation_data = {'Image_Path': image_test, 'Label': label_test, 'Mask_Path': mask_test}
    df_validation = pd.DataFrame(validation_data)
    df_validation.to_csv(os.path.join(cm17_save_dir, 'validation_fold_{0}.csv'.format(counter)), index = False)
    # print (df_validation)
    counter += 1

### CM16 CV SPLITs

In [26]:
# Collect all CM16 Training WSI_paths
cm16_train_file_list = list(set(os.listdir(train_tif_cm16_path)) - set(train_tumor_nea_file_list))
cm16_train_mask_file_list = os.listdir(train_mask_cm16_path)
cm16_test_mask_file_list = os.listdir(test_mask_cm16_path)

normal_list = []
tumor_list = []

normal_mask_list = []
tumor_mask_list = []

for file in cm16_train_file_list:
    mask_file = file.split('.')[0]+'_Mask.tif'
    if mask_file in cm16_train_mask_file_list:
        tumor_list.append(os.path.join(train_tif_cm16_path, file))
        tumor_mask_list.append(os.path.join(train_mask_cm16_path, mask_file))
    else:
        normal_list.append(os.path.join(train_tif_cm16_path, file))
        normal_mask_list.append(0)
        
cm16_train_patient_list = normal_list + tumor_list
cm16_train_patient_mask_list = normal_mask_list + tumor_mask_list
cm16_train_patient_label_list = len(normal_list)*[0] + len(tumor_list)*[1]
print ('Total_number of CM16 training cases with annotations', len(cm16_train_patient_label_list))

Total_number of CM16 training cases with annotations 250


In [27]:
SPLITS = 3
cm16_train_save_dir = os.path.join(save_path, 'cm16_train_cross_val_splits')
if not os.path.exists(cm16_train_save_dir):
    os.makedirs(cm16_train_save_dir)
    
skf = StratifiedKFold(n_splits=SPLITS)
# skf.get_n_splits(cm16_train_patient_list, cm16_train_patient_label_list)
counter = 0
for train_index, test_index in skf.split(cm16_train_patient_list, cm16_train_patient_label_list):
#     print("TRAIN:", train_index, "TEST:", test_index)    
    image_train, image_test = itemgetter(*train_index)(cm16_train_patient_list), itemgetter(*test_index)(cm16_train_patient_list) 
    label_train, label_test = itemgetter(*train_index)(cm16_train_patient_label_list), itemgetter(*test_index)(cm16_train_patient_label_list) 
    mask_train, mask_test = itemgetter(*train_index)(cm16_train_patient_mask_list), itemgetter(*test_index)(cm16_train_patient_mask_list) 
    
    train_data = {'Image_Path': image_train, 'Label': label_train, 'Mask_Path': mask_train}
    df_train = pd.DataFrame(train_data)
    # print (df_train)
    df_train.to_csv(os.path.join(cm16_train_save_dir, 'training_fold_{0}.csv'.format(counter)), index = False)
    validation_data = {'Image_Path': image_test, 'Label': label_test, 'Mask_Path': mask_test}
    df_validation = pd.DataFrame(validation_data)
    df_validation.to_csv(os.path.join(cm16_train_save_dir, 'validation_fold_{0}.csv'.format(counter)), index = False)
    # print (df_validation)
    counter += 1

In [28]:
# Collect all CM16 Testing WSI_paths
cm16_test_stage_labels_df = pd.read_csv(test_reference_cm16_path)
cm16_test_file_list = list(set(os.listdir(test_tif_cm16_path)) - set(test_tumor_nea_file_list))
cm16_test_mask_file_list = os.listdir(test_mask_cm16_path)

normal_list = []
tumor_list = []

normal_mask_list = []
tumor_mask_list = []

for file in cm16_test_file_list:
    mask_file = file.split('.')[0]+'_Mask.tif'
    if mask_file in cm16_test_mask_file_list:
        tumor_list.append(os.path.join(test_tif_cm16_path, file))
        tumor_mask_list.append(os.path.join(test_mask_cm16_path, mask_file))
    else:
        normal_list.append(os.path.join(test_tif_cm16_path, file))
        normal_mask_list.append(0)
        
cm16_test_patient_list = normal_list + tumor_list
cm16_test_patient_mask_list = normal_mask_list + tumor_mask_list
cm16_test_patient_label_list = len(normal_list)*[0] + len(tumor_list)*[1]
print ('Total_number of CM16 test cases with annotations', len(cm16_test_patient_label_list))

Total_number of CM16 test cases with annotations 128


###  CM16 Test SPLITS

In [29]:
SPLITS = 3
cm16_test_save_dir = os.path.join(save_path, 'cm16_test_cross_val_splits')
if not os.path.exists(cm16_test_save_dir):
    os.makedirs(cm16_test_save_dir)
    
skf = StratifiedKFold(n_splits=SPLITS)
# skf.get_n_splits(cm16_train_patient_list, cm16_train_patient_label_list)
counter = 0
for train_index, test_index in skf.split(cm16_test_patient_list, cm16_test_patient_label_list):
#     print("TRAIN:", train_index, "TEST:", test_index)    
    image_train, image_test = itemgetter(*train_index)(cm16_test_patient_list), itemgetter(*test_index)(cm16_test_patient_list) 
    label_train, label_test = itemgetter(*train_index)(cm16_test_patient_label_list), itemgetter(*test_index)(cm16_test_patient_label_list) 
    mask_train, mask_test = itemgetter(*train_index)(cm16_test_patient_mask_list), itemgetter(*test_index)(cm16_test_patient_mask_list) 
    
    train_data = {'Image_Path': image_train, 'Label': label_train, 'Mask_Path': mask_train}
    df_train = pd.DataFrame(train_data)
    # print (df_train)
    df_train.to_csv(os.path.join(cm16_test_save_dir, 'training_fold_{0}.csv'.format(counter)), index = False)
    validation_data = {'Image_Path': image_test, 'Label': label_test, 'Mask_Path': mask_test}
    df_validation = pd.DataFrame(validation_data)
    df_validation.to_csv(os.path.join(cm16_test_save_dir, 'validation_fold_{0}.csv'.format(counter)), index = False)
    # print (df_validation)
    counter += 1

### Combine CM16 Train and CM17 Train

In [14]:
cm16_17_train_save_dir = os.path.join(save_path, 'cm16_17_train_cross_val_splits')
if not os.path.exists(cm16_17_train_save_dir):
    os.makedirs(cm16_17_train_save_dir)
    
cm16_17_train_patient_list = cm17_train_patient_list + cm16_train_patient_list
cm16_17_train_patient_mask_list = cm17_train_patient_mask_list + cm16_train_patient_mask_list
cm16_17_train_patient_label_list = cm17_patient_label_list + cm16_train_patient_label_list

print(len(cm16_17_train_patient_list))

SPLITS = 3
    
skf = StratifiedKFold(n_splits=SPLITS)
counter = 0
for train_index, test_index in skf.split(cm16_17_train_patient_list, cm16_17_train_patient_label_list):
#     print("TRAIN:", train_index, "TEST:", test_index)    
    image_train, image_test = itemgetter(*train_index)(cm16_17_train_patient_list), itemgetter(*test_index)(cm16_17_train_patient_list) 
    label_train, label_test = itemgetter(*train_index)(cm16_17_train_patient_label_list), itemgetter(*test_index)(cm16_17_train_patient_label_list) 
    mask_train, mask_test = itemgetter(*train_index)(cm16_17_train_patient_mask_list), itemgetter(*test_index)(cm16_17_train_patient_mask_list) 
    
    train_data = {'Image_Path': image_train, 'Label': label_train, 'Mask_path': mask_train}
    df_train = pd.DataFrame(train_data)
    # print (df_train)
    df_train.to_csv(os.path.join(cm16_17_train_save_dir, 'training_fold_{0}.csv'.format(counter)), index = False)
    validation_data = {'Image_Path': image_test, 'Label': label_test, 'Mask_Path': mask_test}
    df_validation = pd.DataFrame(validation_data)
    df_validation.to_csv(os.path.join(cm16_17_train_save_dir, 'validation_fold_{0}.csv'.format(counter)), index = False)
    # print (df_validation)
    counter += 1

618


### Combine CM16 Train & CM16 Test

In [15]:
cm16_train_test_save_dir = os.path.join(save_path, 'cm16_train_test_cross_val_splits')
if not os.path.exists(cm16_train_test_save_dir):
    os.makedirs(cm16_train_test_save_dir)
    
cm16_train_test_patient_list = cm16_train_patient_list + cm16_test_patient_list
cm16_train_test_mask_list = cm16_train_patient_mask_list + cm16_test_patient_mask_list
cm16_train_test_label_list = cm16_train_patient_label_list + cm16_test_patient_label_list

print(len(cm16_train_test_patient_list))

SPLITS = 3
    
skf = StratifiedKFold(n_splits=SPLITS)
counter = 0
for train_index, test_index in skf.split(cm16_train_test_patient_list, cm16_train_test_label_list):
#     print("TRAIN:", train_index, "TEST:", test_index)    
    image_train, image_test = itemgetter(*train_index)(cm16_train_test_patient_list), itemgetter(*test_index)(cm16_train_test_patient_list) 
    label_train, label_test = itemgetter(*train_index)(cm16_train_test_label_list), itemgetter(*test_index)(cm16_train_test_label_list) 
    mask_train, mask_test = itemgetter(*train_index)(cm16_train_test_mask_list), itemgetter(*test_index)(cm16_train_test_mask_list) 
    
    train_data = {'Image_Path': image_train, 'Label': label_train, 'Mask_Path': mask_train}
    df_train = pd.DataFrame(train_data)
    # print (df_train)
    df_train.to_csv(os.path.join(cm16_train_test_save_dir, 'training_fold_{0}.csv'.format(counter)), index = False)
    validation_data = {'Image_Path': image_test, 'Label': label_test, 'Mask_Path': mask_test}
    df_validation = pd.DataFrame(validation_data)
    df_validation.to_csv(os.path.join(cm16_train_test_save_dir, 'validation_fold_{0}.csv'.format(counter)), index = False)
    # print (df_validation)
    counter += 1

378


### Combine CM16 Train & CM16 Test and CM17 Train

In [14]:
save_path = '../../data/raw-data'
tumor_type = 'whole'
cm16_train_test_cm17_train_save_dir = os.path.join(save_path, 'train_cross_val_splits_%s'%(tumor_type))
if not os.path.exists(cm16_train_test_cm17_train_save_dir):
    os.makedirs(cm16_train_test_cm17_train_save_dir)
    
cm16_train_test_cm17_patient_list = glob.glob(os.path.join(save_path,'**','*.svs'),recursive=True)
cm16_train_test_cm17_mask_list = glob.glob(os.path.join(save_path,'**','*%s*.tiff'%(tumor_type)),recursive=True)

print(len(cm16_train_test_cm17_patient_list),len(cm16_train_test_cm17_mask_list))

SPLITS =3
    
skf = KFold(n_splits=SPLITS)
counter = 0
for train_index, test_index in skf.split(cm16_train_test_cm17_patient_list, cm16_train_test_cm17_mask_list):
    print("TRAIN:", train_index, "TEST:", test_index)    
    image_train, image_test = itemgetter(*train_index)(cm16_train_test_cm17_patient_list), itemgetter(*test_index)(cm16_train_test_cm17_patient_list) 
    mask_train, mask_test = itemgetter(*train_index)(cm16_train_test_cm17_mask_list), itemgetter(*test_index)(cm16_train_test_cm17_mask_list) 
    
    train_data = {'Image_Path': image_train ,'Mask_Path': mask_train}
    df_train = pd.DataFrame(train_data)
    # print (df_train)
    df_train.to_csv(os.path.join(cm16_train_test_cm17_train_save_dir, 'training_fold_{0}.csv'.format(counter)), index = False)
    validation_data = {'Image_Path': image_test, 'Mask_Path': mask_test}
    df_validation = pd.DataFrame(validation_data)
    df_validation.to_csv(os.path.join(cm16_train_test_cm17_train_save_dir, 'validation_fold_{0}.csv'.format(counter)), index = False)
    # print (df_validation)
    counter += 1

50 50
TRAIN: [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49] TEST: [0 1 2 3 4 5 6 7 8 9]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49] TEST: [10 11 12 13 14 15 16 17 18 19]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49] TEST: [20 21 22 23 24 25 26 27 28 29]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 40 41 42 43 44 45 46 47 48 49] TEST: [30 31 32 33 34 35 36 37 38 39]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] TEST: [40 41 42 43 44 45 46 47 48 49]
